In [1]:
import pandas as pd
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from tensorflow import lite

Using TensorFlow backend.


In [24]:
df=pd.read_excel('GameScore_dataset.xlsx')
df=df.drop(['Unnamed: 0'], axis=1)
properties = list(df.columns.values)
properties.remove('alzheimer')
X = df[properties]
y = df['alzheimer']

#df=df.drop(['attention_time'], axis=1)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [26]:
model = keras.Sequential([
    keras.layers.Dense(9,input_shape=[9,]),
    #keras.layers.Flatten(input_shape=[9,]),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

In [27]:
X_train

,speed,speed_time,memory,memory_time,calculation,calculation_time,attention,attention_time,path_tracing_time
105,17,107,13,191,12,100,17,120,380
20,11,17,12,185,0,99,13,120,541
47,11,23,12,185,4,83,18,120,520
144,48,95,17,146,16,50,30,120,273
5,11,28,11,189,8,73,6,120,471
...,...,...,...,...,...,...,...,...,...
64,13,15,13,188,4,94,8,120,547
15,9,10,12,210,4,92,5,120,573
228,54,115,20,149,16,63,43,120,210
125,47,86,16,155,16,52,59,120,219


In [28]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [29]:
model.fit(X_train, y_train, epochs=50, batch_size=1)

test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)


Epoch 1/50
198/198 [==============================] - 1s 5ms/step - loss: 6.6324 - accuracy: 0.4848
Epoch 2/50
198/198 [==============================] - 0s 922us/step - loss: 0.5140 - accuracy: 0.8081
Epoch 3/50
198/198 [==============================] - 0s 927us/step - loss: 0.4516 - accuracy: 0.84850s - loss: 0.4734 - accuracy: 
Epoch 4/50
198/198 [==============================] - 0s 882us/step - loss: 0.4362 - accuracy: 0.8535
Epoch 5/50
198/198 [==============================] - 0s 906us/step - loss: 0.3651 - accuracy: 0.8939
Epoch 6/50
198/198 [==============================] - 0s 900us/step - loss: 0.3971 - accuracy: 0.8788
Epoch 7/50
198/198 [==============================] - 0s 887us/step - loss: 0.3587 - accuracy: 0.8535
Epoch 8/50
198/198 [==============================] - 0s 967us/step - loss: 0.3199 - accuracy: 0.9091
Epoch 9/50
198/198 [==============================] - 0s 972us/step - loss: 0.3472 - accuracy: 0.8990
Epoch 10/50
198/198 [==============================] 

In [30]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 9)                 90        
_________________________________________________________________
dense_14 (Dense)             (None, 16)                160       
_________________________________________________________________
dense_15 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 17        
Total params: 539
Trainable params: 539
Non-trainable params: 0
_________________________________________________________________


In [31]:
X_test[1:2]['speed']

61    4
Name: speed, dtype: int64

In [35]:
y_pred=model.predict(X_test)
y_pred_class=model.predict_classes(X_test, verbose=0)
yhat_probs = y_pred[:, 0]
yhat_classes = y_pred_class[:, 0]

In [46]:
#from sklearn.metrics import accuracy_score
import sklearn.metrics

In [52]:
accuracy = accuracy_score( y_test, y_pred.round())
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = sklearn.metrics.precision_score(y_test, y_pred.round())
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = sklearn.metrics.recall_score(y_test, y_pred.round())
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = sklearn.metrics.f1_score(y_test, y_pred.round())
print('F1 score: %f' % f1)

Accuracy: 0.900000
Precision: 0.880000
Recall: 0.916667
F1 score: 0.897959


In [42]:
from sklearn.metrics import mean_squared_error
from math import sqrt
NN_mse=mean_squared_error(y_test,y_pred)
print(NN_mse)

0.07843532203602395


In [ ]:
rms_NN = sqrt(mean_squared_error(y_test, y_predicted))
print(rms_NN)

In [ ]:
#saving model by using tf save model
model.save('saved_model') 

In [ ]:
#saving model as pkl
import pickle
weigh= model.get_weights()    
pklfile= "modelweights.pkl"

fpkl= open(pklfile, 'wb')    #Python 3     

pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
print(type(pklfile))
fpkl.close()

In [ ]:
keras_file="linear.h5"
keras.models.save_model(model,keras_file)

In [ ]:
converter= lite.TocoConverter.from_keras_model_file(keras_file)
tflite_model=converter.convert()
open("NN_game_score.tflite", "wb").write(tflite_model)

In [ ]:
model.evaluate_